In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import binom, binom_test
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import math
from sklearn.linear_model import LogisticRegression
# import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
predicted_street = pd.read_pickle(r'file1')
predicted_pension = pd.read_pickle(r'file2')
predicted_nonpension = pd.read_pickle(r'file3')

actual_street =  pd.read_pickle(r"file4")
actual_pension =  pd.read_pickle(r"file5")
actual_nonpension =  pd.read_pickle(r"file6")

In [ ]:
def df_prep(predicted, actual):
    predicted = predicted.iloc[:,1:]
    predicted['CONTRACT_REF_NO'] = predicted['CONTRACT_REF_NO'].str.split('_', 1).str[0]
    predicted = predicted.rename(columns = {'CONTRACT_REF_NO':'CONTRACT_REF_NO_raw'})
    df_merged = predicted.merge(actual, on=['CONTRACT_REF_NO_raw'])
    df_merged = df_merged.rename(columns = {'CONTRACT_REF_NO_raw':'obs','final_pd':'pd','bad_flag_y':'bad_flag'})
    return df_merged

In [ ]:
df_merged_street = df_prep(predicted_street, actual_street)
df_merged_pension = df_prep(predicted_pension, actual_pension)
df_merged_nonpension = df_prep(predicted_nonpension, actual_nonpension)

## Assumption 2 — Linearity of independent variables and log-odds

One of the critical assumptions of logistic regression is that the relationship between the logit (aka log-odds) of the outcome and each continuous independent variable is linear.

In [ ]:
continuous_var = ['pd']

def assump2(df_merged):
    # Add logit transform interaction terms (natural log) for continuous variables e.g.. Age * Log(Age)
    for var in continuous_var:
        df_merged[f'{var}:Log_{var}'] = df_merged[var].apply(lambda x: x * np.log(x))

    # Keep columns related to continuous variables
    cols_to_keep = continuous_var + df_merged.columns.tolist()[-len(continuous_var):]

    # Redefining variables to include interaction terms
    X_lt = df_merged[cols_to_keep]
    X_lt_constant = sm.add_constant(X_lt, prepend=False)
    y = df_merged['bad_flag']
    return X_lt_constant, y

In [ ]:
X_lt_constant_street, y_street = assump2(df_merged_street)
X_lt_constant_pension, y_pension = assump2(df_merged_pension)
X_lt_constant_nonpension, y_nonpension = assump2(df_merged_nonpension)

In [ ]:
# STREET
# Building model and fit the data (using statsmodel's Logit)
logit_results_street = GLM(y_street, X_lt_constant_street, family=families.Binomial()).fit()
# Display summary results
print(logit_results_street.summary())

In [ ]:
#PENSION
# Building model and fit the data (using statsmodel's Logit)
logit_results_pension = GLM(y_pension, X_lt_constant_pension, family=families.Binomial()).fit()
# Display summary results
print(logit_results_pension.summary())

In [ ]:
#NON-PENSION
# Building model and fit the data (using statsmodel's Logit)
logit_results_nonpension = GLM(y_nonpension, X_lt_constant_nonpension, family=families.Binomial()).fit()
# Display summary results
print(logit_results_nonpension.summary())

**pd:Log_pd**  is statistically significant (i.e., p≤0.05), indicating the presence of non-linearity between Lod_pd and the logit. 

One solution is to perform transformations by incorporating higher-order polynomial terms to capture the non-linearity **(e.g., Log_pd²)**.

In [ ]:
df_merged_nonpension.columns

In [ ]:
df_merged_street = df_merged_street[['rm_3_sum_Total_12', 'mS_ever_01d_all_all_09',
       'vm_010_o_prin_01d_all_all_12', 'Age_gender', 'Sektor', 'pd']]
df_merged_pension = df_merged_pension[['vm_100_o_sal_sal_09', 'mS_ever_15d_all_all_06', 'Age_gender', 'Region',
       'pd']]
df_merged_nonpension = df_merged_nonpension[['rm_3_sum_Total_12', 'mS_ever_15d_all_all_12',
       'vm_010_o_tota_01d_all_all_12', 'Age_gender', 'Ministry', 'pd']]

In [ ]:
# Define dependent and independent variables
# X_street = df_merged_street[['pd','Age_gender','Sektor','rm_3_sum_Total_12','vm_010_o_prin_01d_all_all_12','mS_ever_01d_all_all_09']]
# X_pension = df_merged_pension[['pd','Age_gender_pred_prob','Region_pred_prob','mS_ever_15d_all_all_06_pd_pred_prob','vm_100_o_sal_sal_09_pred_prob']]
# X_nonpension = df_merged_nonpension[['pd','Age_gender_pred_prob','Ministry_pd_pred_prob','rm_3_sum_Total_12_pred_prob','mS_ever_15d_all_all_12_pd_pred_prob','vm_010_o_tota_01d_all_all_12_pd']]
X_street = df_merged_street.copy()
X_pension = df_merged_pension.rename(columns = {'Age_gender':'Age_gender_pred_prob','mS_ever_15d_all_all_06':'mS_ever_15d_all_all_06_pd_pred_prob','vm_100_o_sal_sal_09':'vm_100_o_sal_sal_09_pred_prob'})
X_nonpension = df_merged_nonpension.rename(columns = {'Age_gender':'Age_gender_pred_prob','Ministry':'Ministry_pd_pred_prob','rm_3_sum_Total_12':'rm_3_sum_Total_12_pred_prob','mS_ever_15d_all_all_12':'mS_ever_15d_all_all_12_pd_pred_prob','vm_010_o_tota_01d_all_all_12':'vm_010_o_tota_01d_all_all_12_pd'})

In [ ]:
X_constant_street = sm.add_constant(X_street, prepend=False)
X_constant_pension = sm.add_constant(X_pension, prepend=False)
X_constant_nonpension = sm.add_constant(X_nonpension, prepend=False)

In [ ]:
def logit_results(X_constant, y):
    # Re-running logistic regression on the original set of X and y variables
    logit_results = GLM(y, X_constant, family=families.Binomial()).fit()
    predicted = logit_results.predict(X_constant)

    # Getting log odds values
    log_odds = np.log(predicted / (1 - predicted))

    # Visualize predictor variable vs logit values for Age
    plt.scatter(x=X_constant['pd'].values, y=log_odds)
    plt.xlabel("pd")
    plt.ylabel("Log-odds")
    plt.show()

In [ ]:
logit_results(X_constant_street, y_street)

In [ ]:
logit_results(X_constant_pension, y_pension)

In [ ]:
logit_results(X_constant_nonpension, y_nonpension)

The above scatter plot shows a clear non-linear pattern of **pd** vs. the log-odds, thereby implying that the assumption of logit linearity is violated.

## Let's try only with pd

In [ ]:
X_pd_street = df_merged_street['pd']
X_pd_pension = df_merged_pension['pd']
X_pd_nonpension = df_merged_nonpension['pd']

In [ ]:
X_pd_constant_street = sm.add_constant(X_pd_street, prepend=False)
X_pd_constant_pension = sm.add_constant(X_pd_pension, prepend=False)
X_pd_constant_nonpension = sm.add_constant(X_pd_nonpension, prepend=False)

In [ ]:
logit_results(X_pd_constant_street, y_street)

In [ ]:
logit_results(X_pd_constant_pension, y_pension)

In [ ]:
logit_results(X_pd_constant_nonpension, y_nonpension)

## Assumption 3— No strongly influential outliers

Logistic regression assumes that there are no highly influential outlier data points, as they distort the outcome and accuracy of the model.

Note that not all outliers are influential observations. Rather, outliers have the potential to be influential. To assess this assumption, we need to check whether both criteria are satisfied, i.e., influential and outlier.

### Influence

We can use Cook’s Distance to determine the influence of a data point, and it is calculated based on its residual and leverage. It summarizes the changes in the regression model when that particular (ith) observation is removed.

One standard threshold is 4/N (where N = number of observations), meaning that observations with Cook’s Distance > 4/N are deemed as influential.

In [ ]:
# Use GLM method for logreg here so that we can retrieve the influence measures
logit_model_street = GLM(y_street, X_constant_street, family=families.Binomial())
logit_results_street= logit_model_street.fit()
print("---------------------------------STREET------------------------------------")
print(logit_results_street.summary())

In [ ]:
# Use GLM method for logreg here so that we can retrieve the influence measures
logit_model_pension = GLM(y_pension, X_constant_pension, family=families.Binomial())
logit_results_pension= logit_model_pension.fit()
print("---------------------------------PENSION---------------------------------")
print(logit_results_pension.summary())

In [ ]:
# Use GLM method for logreg here so that we can retrieve the influence measures
logit_model_nonpension = GLM(y_nonpension, X_constant_nonpension, family=families.Binomial())
logit_results_nonpension= logit_model_nonpension.fit()
print("-------------------------------NON-PENSION-------------------------------")
print(logit_results_nonpension.summary())

In [ ]:
def assump3(logit_results, X):
    # Get influence measures
    influence = logit_results.get_influence()

    # Obtain summary df of influence measures
    summ_df = influence.summary_frame()
    diagnosis_df = summ_df.loc[:,['cooks_d']]

    # Append absolute standardized residual values
    diagnosis_df['std_resid'] = stats.zscore(logit_results.resid_pearson)
    diagnosis_df['std_resid'] = diagnosis_df.loc[:,'std_resid'].apply(lambda x: np.abs(x))

    # Sort by Cook's Distance
    diagnosis_df.sort_values("cooks_d", ascending=False)
    display(diagnosis_df)

    # Set Cook's distance threshold
    cook_threshold = 4 / len(X)
    print(f"Threshold for Cook Distance = {cook_threshold}")

    # Plot influence measures (Cook's distance)
    fig = influence.plot_index(y_var="cooks", threshold=cook_threshold)
    plt.axhline(y=cook_threshold, ls="--", color='red')
    fig.tight_layout(pad=2)

    # Find number of observations that exceed Cook's distance threshold
    outliers = diagnosis_df[diagnosis_df['cooks_d'] > cook_threshold]
    prop_outliers = round(100*(len(outliers) / len(X)),1)
    print(f'Proportion of data points that are highly influential = {prop_outliers}%')

    # Find number of observations which are BOTH outlier (std dev > 3) and highly influential
    extreme = diagnosis_df[(diagnosis_df['cooks_d'] > cook_threshold) & 
                        (diagnosis_df['std_resid'] > 3)]
    prop_extreme = round(100*(len(extreme) / len(X)),1)
    print(f'Proportion of highly influential outliers = {prop_extreme}%')

    # Display top 5 most influential outliers
    display(extreme.sort_values("cooks_d", ascending=False).head())

In [ ]:
assump3(logit_results_street, X_street)

In [ ]:
assump3(logit_results_pension, X_pension)

In [ ]:
assump3(logit_results_nonpension, X_nonpension)

## Let's try only with pd

In [ ]:
# Use GLM method for logreg here so that we can retrieve the influence measures
logit_model_pd_street = GLM(y_street, X_pd_constant_street, family=families.Binomial())
logit_results_pd_street = logit_model_pd_street.fit()
print(logit_results_pd_street.summary())

In [ ]:
# Use GLM method for logreg here so that we can retrieve the influence measures
logit_model_pd_pension = GLM(y_pension, X_pd_constant_pension, family=families.Binomial())
logit_results_pd_pension = logit_model_pd_pension.fit()
print(logit_results_pd_pension.summary())

In [ ]:
# Use GLM method for logreg here so that we can retrieve the influence measures
logit_model_pd_nonpension = GLM(y_nonpension, X_pd_constant_nonpension, family=families.Binomial())
logit_results_pd_nonpension = logit_model_pd_nonpension.fit()
print(logit_results_pd_nonpension.summary())

In [ ]:
assump3(logit_results_pd_street, X_pd_constant_street)

In [ ]:
# Deep dive into index 3508 (extreme outlier)
print("Prob: "+ str(X_pd_constant_street.pd.iloc[89296]))
print("Bad flag: "+ str(y_street.iloc[89296]))  # 1 = Default

In [ ]:
# Deep dive into index 3405 (extreme outlier)
print("Prob: "+ str(X_pd_constant_street.pd.iloc[12586]))
print("Bad flag: "+ str(y_street.iloc[12586]))  # 1 = Default

In [ ]:
# Deep dive into index 652 (extreme outlier)
print("Prob: "+ str(X_pd_constant_street.pd.iloc[74892]))
print("Bad flag: "+ str(y_street.iloc[74892]))  # 1 = Default

In [ ]:
assump3(logit_results_pd_pension, X_pd_constant_pension)

In [ ]:
# Deep dive into index 46000 (extreme outlier)
print("Prob: "+ str(X_pd_constant_pension.pd.iloc[115418]))
print("Bad flag: "+ str(y_pension.iloc[115418]))  # 1 = Default

In [ ]:
# Deep dive into index 51774 (extreme outlier)
print("Prob: "+ str(X_pd_constant_pension.pd.iloc[68296]))
print("Bad flag: "+ str(y_pension.iloc[68296]))  # 1 = Default

In [ ]:
# Deep dive into index 58310 (extreme outlier)
print("Prob: "+ str(X_pd_constant_pension.pd.iloc[69903]))
print("Bad flag: "+ str(y_pension.iloc[69903]))  # 1 = Default

In [ ]:
assump3(logit_results_pd_nonpension, X_pd_constant_nonpension)

In [ ]:
# Deep dive into index 58672 (extreme outlier)
print("Prob: "+ str(X_pd_constant_nonpension.pd.iloc[46932]))
print("Bad flag: "+ str(y_nonpension.iloc[46932]))  # 1 = Default

In [ ]:
# Deep dive into index 23751 (extreme outlier)
print("Prob: "+ str(X_pd_constant_nonpension.pd.iloc[34336]))
print("Bad flag: "+ str(y_nonpension.iloc[34336]))  # 1 = Default

In [ ]:
# Deep dive into index 36100 (extreme outlier)
print("Prob: "+ str(X_pd_constant_nonpension.pd.iloc[201590]))
print("Bad flag: "+ str(y_nonpension.iloc[201590]))  # 1 = Default

### It is important to note that for data points with relative high Cook's distances, it does not automatically mean that it should be immediately removed from the dataset. It is essentially an indicator to highlight which data points are worth looking deeper into, to understand whether they are true anomalies or not
In practice, an assessment of “large” values is a judgement call based on experience and the particular set of data being analyzed.
In addition, based on our pre-defined threshold (4/N), only 5% of the points are in the outlier zone, which is small as well. The issue comes when there is a significant number of data points classified as outliers.
The management of outliers is outside the scope of this demo

## Assumption 4 — Absence of Multicollinearity

it  is okay

## Assumption 5— Independence of observations

The observations must be independent of each other, i.e., they should not come from repeated or paired data. This means that each observation is not influenced by or related to the rest of the observations.

### Check residuals series

In [ ]:
def residual_series(logit_results, X):
    # Generate residual series plot
    fig = plt.figure(figsize=(8,5))
    ax = fig.add_subplot(111, title="Residual Series Plot",
                        xlabel="Index Number", ylabel="Deviance Residuals")

    # ax.plot(X_pd.index.tolist(), stats.zscore(logit_results.resid_pearson))
    ax.plot(X.index.tolist(), stats.zscore(logit_results.resid_deviance))
    plt.axhline(y=0, ls="--", color='red')

In [ ]:
residual_series(logit_results_pd_street, X_pd_street)

In [ ]:
residual_series(logit_results_pd_pension, X_pd_pension)

In [ ]:
residual_series(logit_results_pd_nonpension, X_pd_nonpension)

Since the residuals in the plot above appear to be randomly scattered around the centerline of zero, we can infer (visually) that the assumption is satisfied.

# All assumptions are satisfied